In [1]:
import numpy as np
from math import pi
import datetime as dt
import pandas as pd
from bokeh.plotting import figure, output_file, show
from bokeh.models import LinearAxis, LogAxis, Range1d
from scipy.fftpack import fft, fftfreq, fftshift, rfft
output_file('Fourier.html')

In [2]:
tick = pd.read_csv('RIM6_4.trd', usecols=(0, 1, 2, 3, 4, 5, 6),
                   parse_dates={'DateTime': [0,1]}, index_col=False)

In [3]:
tick = tick.rename(columns={'<LAST>': 'Price', '<VOL>': 'Vol', '<DIRECTION>': 'Dir'})

In [4]:
tick.ix[tick.Dir=='Sell', 'VolSell'] = tick.Vol

In [5]:
tick.ix[tick.Dir=='Buy', 'VolBuy'] = tick.Vol

In [6]:
Nticks=500 #Количество тиков для бара и ресэмплинг

In [7]:
# main=pd.DataFrame(index=np.arange(0, (len(tr.index)/Nticks+Nticks)),
#                 columns=('DateTime', 'open', 'high', 'low', 'close', 'vBuy', 'vSell'))
rows_list=[]
for i in range(0, len(tick.index), Nticks):
    dict1={}
#    srez = tr.iloc[i:i+Nticks]
    op=tick.Price.iloc[i]
    hi=tick.Price.iloc[i:i+Nticks].max()
    lo=tick.Price.iloc[i:i+Nticks].min()
    if i+Nticks<len(tick.index):
        cl=tick.loc[i+Nticks-1].Price
    else:
        cl=tick.loc[len(tick.index)-1].Price
    
    vBuy=tick.VolBuy.iloc[i:i+Nticks].sum()
    vSell=tick.VolSell.iloc[i:i+Nticks].sum()
    nBuy=tick.VolBuy.iloc[i:i+Nticks].count()
    nSell=tick.VolSell.iloc[i:i+Nticks].count()
    tClose=tick.DateTime.iloc[i:i+Nticks].max()
    dtime=tick.DateTime.iloc[i:i+Nticks].max()-tick.DateTime.iloc[i:i+Nticks].min()
    dict1.update({'DateTime': tClose, 'open': op, 'high': hi,
                 'low': lo, 'close': cl, 'TickBuy': nBuy,
                'VolBuy': vBuy, 'TickSell': nSell, 'VolSell': vSell, 'DeltaTime': dtime})
    rows_list.append(dict1)
    
res=pd.DataFrame(rows_list)
res['Dsec']=res['DeltaTime'].dt.total_seconds()
res['TSpeedBuy']=res['TickBuy']/res['Dsec']
res['TSpeedSell']=res['TickSell']/res['Dsec']
res['VSpeedBuy']=res['VolBuy']/res['Dsec']
res['VSpeedSell']=res['VolSell']/res['Dsec']
res['OTO']=((res['TickBuy']-res['TickSell'])/(res['TickBuy']+res['TickSell'])*0.65
            + (res['VolBuy']-res['VolSell'])/(res['VolBuy']+res['VolSell'])*0.35)*100.0
res['OTON']=((res['TickBuy']*res['VolBuy']-res['TickSell']*res['VolSell'])/
             (res['TickBuy']*res['VolBuy']+res['TickSell']*res['VolSell']))*100.0
res['Delta']=res['VolBuy']-res['VolSell']
res['CumDelta']=res['Delta'].cumsum()

In [8]:
res

,DateTime,DeltaTime,TickBuy,TickSell,VolBuy,VolSell,close,high,low,open,Dsec,TSpeedBuy,TSpeedSell,VSpeedBuy,VSpeedSell,OTO,OTON,Delta,CumDelta
0,2016-04-20 10:00:01,00:00:01,96,404,154.0,1535.0,90950,91860,90730,91860,1.0,96.000000,404.000000,154.000000,1535.000000,-68.657525,-95.343065,-1381.0,-1381.0
1,2016-04-20 10:00:02,00:00:01,195,305,287.0,510.0,90820,90920,90650,90900,1.0,195.000000,305.000000,287.000000,510.000000,-24.092974,-47.081767,-223.0,-1604.0
2,2016-04-20 10:00:03,00:00:01,192,308,405.0,696.0,90640,90890,90630,90830,1.0,192.000000,308.000000,405.000000,696.000000,-24.330681,-46.763063,-291.0,-1895.0
3,2016-04-20 10:00:05,00:00:02,234,266,461.0,951.0,90640,90680,90550,90640,2.0,117.000000,133.000000,230.500000,475.500000,-16.305892,-40.209511,-490.0,-2385.0
4,2016-04-20 10:00:07,00:00:02,247,253,408.0,384.0,90630,90690,90590,90630,2.0,123.500000,126.500000,204.000000,192.000000,0.280606,1.830969,24.0,-2361.0
5,2016-04-20 10:00:10,00:00:03,265,235,472.0,361.0,90650,90690,90590,90640,3.0,88.333333,78.333333,157.333333,120.333333,8.563866,19.172046,111.0,-2250.0
6,2016-04-20 10:00:13,00:00:03,244,256,408.0,495.0,90640,90700,90560,90650,3.0,81.333333,85.333333,136.000000,165.000000,-4.932093,-12.006788,-87.0,-2337.0
7,2016-04-20 10:00:20,00:00:07,233,267,449.0,759.0,90700,90730,90620,90640,7.0,33.285714,38.142857,64.142857,108.428571,-13.401788,-31.905490,-310.0,-2647.0
8,2016-04-20 10:00:28,00:00:08,268,232,777.0,808.0,90890,90890,90700,90700,8.0,33.500000,29.000000,97.125000,101.000000,3.995457,5.251559,-31.0,-2678.0
9,2016-04-20 10:00:34,00:00:06,264,236,643.0,534.0,90940,90960,90840,90890,6.0,44.000000,39.333333,107.166667,89.000000,6.881291,14.784161,109.0,-2569.0


In [9]:
# res = res.set_index('DateTime')
mids = (res.open + res.close)/2
spans = abs(res.close - res.open)
# print res, mids, spans
inc = res.close > res.open
dec = res.open > res.close
#w = 12*60*60
w = 0.5
# print res.index[inc] , mids, spans, inc, dec
resfft = rfft(res[-128:].OTON)
resfftfreq = fftfreq(len(resfft))

In [10]:
p = figure(x_range=(0, len(res.index)), plot_width=800, plot_height=600, title='RIM6 CandleStick')
p.y_range=Range1d(start=90500, end=95000)
p.xaxis.major_label_orientation = pi/4
p.grid.grid_line_alpha = 0.3
p.segment(res.index.values, res.high, res.index.values, res.low, color='black')
p.rect(res.index[inc], mids[inc], w, spans[inc], fill_color='#6CF350', line_color='green')
p.rect(res.index[dec], mids[dec], w, spans[dec], fill_color='#FF0000', line_color='red')
p.extra_y_ranges = {}
p.extra_y_ranges['TickSpeed'] = Range1d(start=0, end=100)
p.extra_y_ranges['OTO'] = Range1d(start=-60, end=60)
p.extra_y_ranges['OTON'] = Range1d(start=-60, end=60)
p.extra_y_ranges['CumDelta'] = Range1d(start=-3000, end=52000)
#p.line(res.index, res.TSpeedBuy, color='blue', y_range_name='TickSpeed', alpha=0.5)
p.line(res.index, res.TSpeedBuy.rolling(window=4).median(), color='green', y_range_name='TickSpeed', alpha=0.8)
#p.line(res.index, res.TSpeedSell, color='red', y_range_name='TickSpeed', alpha=0.5)
p.line(res.index, res.TSpeedSell.rolling(window=9).median(), color='red', y_range_name='TickSpeed', alpha=0.8)
p.line(res.index, res.OTO.rolling(window=15).mean(), color='brown', y_range_name='OTO', alpha=0.9)
p.line(res.index, res.OTON.rolling(window=15).mean(), color='blue', y_range_name='OTON', alpha=0.9)
p.line(res.index, res.CumDelta, color='black', y_range_name='CumDelta', alpha=0.9)
p.add_layout(LogAxis(y_range_name='TickSpeed'), 'left')
p.add_layout(LinearAxis(y_range_name='OTO'), 'left')
p.add_layout(LinearAxis(y_range_name='OTON'), 'left')
p.add_layout(LinearAxis(y_range_name='CumDelta'), 'left')
#show(p)
l=figure(plot_width=800, plot_height=600, title='FFT')
l.line(resfftfreq, resfft)
m = gridplot([[p, l]])

show(m)

NameError: name 'gridplot' is not defined